In [4]:
import multiprocessing
import easyocr
import requests
from io import BytesIO
from PIL import Image
import torch
torch.backends.mps.is_available = lambda: False  # Disable MPS explicitly
multiprocessing.set_start_method("spawn", force=True)
reader = easyocr.Reader(['en'], gpu = False)
def ocr_img_from_url(img_url):
    try:
        response = requests.get(img_url, stream=True)
        if response.status_code != 200:
            print(f"Failed to fetch: {img_url} (Status: {response.status_code})")
            return ""

        # Read image as bytes
        img_bytes = response.content
        
        # Use EasyOCR directly with bytes
        result = reader.readtext(img_bytes)

        # Extract text with high confidence
        text_result = " ".join(text for _, text, prob in result if prob > 0.5)

        return text_result if text_result else "(No text found)"

    except Exception as e:
        print(f"Error processing {img_url}: {e}")
        return ""

Using CPU. Note: This module is much faster with a GPU.


In [5]:
print(torch.device("cpu"))
print(torch.cuda.is_available())  # Should be False
print(torch.backends.mps.is_available())  # Should be False

cpu
False
False


In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('Coin.csv')
df = df[['id', 'address', 'name', 'symbol', 'description', 'mediumImageUrl', 'mimeType']]
df = df.dropna(subset='mediumImageUrl')

In [ ]:
df['img_text'] = df['mediumImageUrl'].apply(lambda x: ocr_img_from_url(x))

In [11]:
import pickle
with open ('ocr.pk', 'wb') as f:
    pickle.dump(reader, f)

AttributeError: Can't pickle local object '_add_module_to_qconfig_obs_ctr.<locals>.get_factory_kwargs_based_on_module_device'

In [12]:
easyocr.__version__

'1.7.2'